In [1]:
from io import open
import glob
import os
import matplotlib.pyplot as plt


def find_files(path): return glob.glob(path)

print(find_files('assignment-07数据集/data/names/*.txt'))

['assignment-07数据集/data/names\\Arabic.txt', 'assignment-07数据集/data/names\\Chinese.txt', 'assignment-07数据集/data/names\\Czech.txt', 'assignment-07数据集/data/names\\Dutch.txt', 'assignment-07数据集/data/names\\English.txt', 'assignment-07数据集/data/names\\French.txt', 'assignment-07数据集/data/names\\German.txt', 'assignment-07数据集/data/names\\Greek.txt', 'assignment-07数据集/data/names\\Irish.txt', 'assignment-07数据集/data/names\\Italian.txt', 'assignment-07数据集/data/names\\Japanese.txt', 'assignment-07数据集/data/names\\Korean.txt', 'assignment-07数据集/data/names\\Polish.txt', 'assignment-07数据集/data/names\\Portuguese.txt', 'assignment-07数据集/data/names\\Russian.txt', 'assignment-07数据集/data/names\\Scottish.txt', 'assignment-07数据集/data/names\\Spanish.txt', 'assignment-07数据集/data/names\\Vietnamese.txt']


In [2]:
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicode_2_Ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicode_2_Ascii('Ślusàrski'))

Slusarski


In [51]:
print(all_letters)
print(len(all_letters))

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .,;'
57


In [17]:
# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def read_lines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicode_2_Ascii(line) for line in lines]

for filename in find_files('assignment-07数据集/data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = read_lines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

In [19]:
print(len(all_categories))

18


In [43]:
category_lines['Chinese'][:10]

['Ang', 'AuYong', 'Bai', 'Ban', 'Bao', 'Bei', 'Bian', 'Bui', 'Cai', 'Cao']

In [77]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letter_to_index(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letter_to_tensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letter_to_index(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def line_to_tensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letter_to_index(letter)] = 1
    return tensor

print(letter_to_index('J'))

print(line_to_tensor('Jones').size())

35
torch.Size([5, 1, 57])


In [78]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(RNN,self).__init__()
        
        self.hidden_size=hidden_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self,input,hidden):
        combined = torch.cat((input,hidden),1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output,hidden
    
    def initHidden(self):
        return torch.zeros(1,self.hidden_size)
    
n_hidden=128

In [79]:
rnn = RNN(n_letters,n_hidden,n_categories)

In [83]:
input = letter_to_tensor('A')
hidden = torch.zeros(1,n_hidden)

output, next_hidden = rnn(input,hidden)

In [84]:
output

tensor([[-2.7949, -2.9423, -2.8141, -2.8904, -2.8186, -2.7994, -2.9244, -2.9434,
         -2.9052, -2.9375, -3.0031, -2.8618, -2.9484, -2.8230, -2.8814, -2.8478,
         -2.9250, -3.0034]], grad_fn=<LogSoftmaxBackward>)

In [85]:
next_hidden

tensor([[ 0.0399,  0.0012,  0.0448,  0.0186,  0.0153, -0.0354,  0.0441, -0.0342,
          0.0803, -0.0449,  0.0590, -0.0602, -0.0437,  0.0662,  0.1126, -0.0263,
         -0.0029,  0.0970,  0.0912,  0.0618,  0.1121,  0.0340, -0.0106,  0.0867,
         -0.0240,  0.0070, -0.0854, -0.0493,  0.0371,  0.0114,  0.0075, -0.0102,
         -0.0365, -0.0734,  0.0096,  0.0723,  0.0448, -0.1012, -0.0299, -0.0238,
          0.0990,  0.0005, -0.0695, -0.0735, -0.0236,  0.0952,  0.1209,  0.0069,
         -0.0061,  0.0548, -0.0762, -0.0329,  0.0056,  0.0542,  0.1066,  0.0106,
         -0.0858, -0.0056,  0.0626,  0.0301, -0.0341, -0.0362, -0.0234, -0.0371,
          0.0072,  0.0180,  0.0842, -0.1106, -0.0092, -0.0037,  0.0033, -0.0034,
         -0.0369,  0.0515,  0.0089,  0.0646,  0.0249, -0.0865, -0.1179, -0.0149,
         -0.0029,  0.0776, -0.0748, -0.0394,  0.1210, -0.0126,  0.0088,  0.0181,
         -0.0608, -0.0764,  0.0004,  0.0365,  0.0180,  0.0406,  0.0437, -0.0506,
          0.0730,  0.0135,  

In [89]:
input=line_to_tensor('Albert')
hidden = torch.zeros(1,n_hidden)

output,next_hidden = rnn(input[0],hidden)
print(output)

tensor([[-2.7949, -2.9423, -2.8141, -2.8904, -2.8186, -2.7994, -2.9244, -2.9434,
         -2.9052, -2.9375, -3.0031, -2.8618, -2.9484, -2.8230, -2.8814, -2.8478,
         -2.9250, -3.0034]], grad_fn=<LogSoftmaxBackward>)


In [91]:
def category_from_output(output):
    top_n,top_i=output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i],category_i

In [92]:
print(category_from_output(output))

('Arabic', 0)


In [95]:
import random

def sample(l):
    return l[random.randint(0,len(l) - 1)]

def sample_training():
    category = sample(all_categories)
    line = sample(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)],dtype=torch.long)
    line_tensor = line_to_tensor(line)
    return category, line, category_tensor, line_tensor

for  i in range(10):
    category, line, category_tensor, line_tensor= sample_training()
    print('category =', category, '/ line =',line)

category = English / line = Hamnett
category = Italian / line = Breda
category = Czech / line = Klein
category = Irish / line = O'Rourke
category = German / line = Nussenbaum
category = Korean / line = Jong
category = Arabic / line = Fakhoury
category = Vietnamese / line = Dao
category = Irish / line = Dubhshlaine
category = Japanese / line = Murkami


In [148]:
sample(category_lines[category])

'Agroskin'

In [152]:
category

'Russian'

In [163]:
category = sample(all_categories)
print(category)

English


In [96]:
criterion = nn.CrossEntropyLoss()

In [116]:
learning_rate = 0.005  #If it is too big, it may explode. If it is too small, it may not learn
def train(category_tensor,lien_tensor):
    hidden = rnn.initHidden()
    
    rnn.zero_grad()
    
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i],hidden)
    
    loss = criterion(output,category_tensor)
    loss.backward()
    
    #Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-learning_rate,p.grad.data)
    
    return output, loss.item()
    

In [121]:
import time
import math

n_iters = 2000 # this number can be bigger from 1000
print_every = 500
plot_every = 100

# Keep track of losses for plotting
current_loss = 0
all_losses = []

def time_since(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m,s)

start = time.time()

for iter  in range(1,n_iters + 1):
    category, line, category_tensor, line_tensor = sample_training()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss
    
    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = category_from_output(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter,iter/n_iters * 100,time_since(start),loss,line,guess,correct))
    
    #Add current loss avg to list  of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss =0

500 25% (0m 1s) 2.7699 Boyle / German ✗ (Scottish)
1000 50% (0m 3s) 2.3262 Spicka / Czech ✓
1500 75% (0m 4s) 2.9004 Dziadik / Polish ✗ (Czech)
2000 100% (0m 6s) 1.8729 Nuryshev / Russian ✓


In [107]:
time.time()

1583428005.278318

In [109]:
start = time.time()

In [122]:
all_losses

[2.615960884690285,
 2.6196786630153657,
 2.5631195491552354,
 2.519955166578293,
 2.553340828418732,
 2.4462936800718307,
 2.561978155970573,
 2.566467969417572,
 2.4661015629768372,
 2.5275044405460356,
 2.428143560886383,
 2.362739366889,
 2.4081400537490847,
 2.4935964381694795,
 2.4749242627620696,
 2.337780596613884,
 2.2551531057059764,
 2.4216844135522844,
 2.524084583520889,
 2.284292483925819]

In [123]:
%matplotlib inline

In [125]:
#plt.plot(all_losses)

In [146]:
def evaluate(line_tensor):
    hidden = rnn.initHidden()
    
    for i in range(line_tensor.size()[0]):
        output,hidden = rnn(line_tensor[i],hidden)
        
    return output

def predict(input_line,n_predictions=3):
    print('\n %s' % input_line)
    with torch.no_grad():
        output = evaluate(line_to_tensor(input_line))
        
        #Get top N categories
        topv,topi = output.topk(n_predictions,1,True)
        predictions = []
        
        for i in range(n_predictions):
            value=topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value,all_categories[category_index]))
            predictions.append([value,all_categories[category_index]])

predict('Dovesky')
predict('Jackson')
predict('Satoshi')
predict('Jerry')


 Dovesky
(-2.08) Russian
(-2.21) Polish
(-2.28) Scottish

 Jackson
(-1.79) Scottish
(-2.17) Russian
(-2.25) Irish

 Satoshi
(-2.13) Polish
(-2.25) Italian
(-2.35) Japanese

 Jerry
(-2.40) Arabic
(-2.46) Czech
(-2.47) Spanish
